In [30]:
import pandas as pd
import re
import seaborn as sbn
import matplotlib.pyplot as plt
import emoji
import string
from emoticon import emoticon
from cleantext import clean
from deep_translator import (GoogleTranslator)
import csv

# from emoticon_lib import emoticon_lib


# Read data
df = pd.read_csv('cyberbullying_tweets.csv')
#show first 25 rows of data
print(df.head(2).to_markdown())

|    | tweet_text                                                                                                          | cyberbullying_type   |
|---:|:--------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc | not_cyberbullying    |


In [31]:
#EDA
# print(df.info())
# print(df.describe())
# print(df.isna().sum())
# print(df.columns.tolist())
# print(df.cyberbullying_type.unique())
# print(df.shape)
# print('Duplicated data num:', df.duplicated().sum())

# word cloud

# print(df[df.duplicated()==True].to_markdown())


In [ ]:
#Remove duplication
# print(df.describe)
# df =df.drop_duplicates(subset='tweet_text', keep="last")
 ############
#Change emojis and emoticon to text(in html code too)
# df['tweet_text'] = df['tweet_text'].apply(lambda e: emoji.demojize(e, language="en"))



# ========================UNDER CONSTRUCTION====================================
# Remove emoticons :/
# https://pypi.org/project/emoticon/

# # Remove RT, #, URL and @:
# df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"(RT)|(:)|(@)|(?:\@|\#|https?|http?|www)\S+", "", e))

#Remove URL & email
# def remove_url_email(df):
    # df['tweet_text'] = df['tweet_text'].apply(lambda e: clean(e, no_urls='True', no_emails='True', replace_with_url='', replace_with_email=''))


#translate tweets
# df['tweet_text'] = df['tweet_text'].apply(lambda e: GoogleTranslator(source='auto', target='en').translate(text=e))

# def t(e):
#     s = GoogleTranslator(source='auto', target='en').translate(text=e)
#     print(s)

# df['tweet_text'].apply(lambda e: t(e))


# EDA to show mssing value
# #Remove missing values in tweet_text after data cleaning
# df = df[~df['tweet_text'].apply(str.isspace())]



#Remove html tags beautiful soup


# ========================UNDER CONSTRUCTION====================================


# print(df.head(100).to_markdown())

#Remove stopwords




# tokenization

# lemmatization

#stemming
